# Fine-Tuninge ModernBERT using GoEmotions dataset

In [ ]:
import numpy as np
import random
import torch
from transformers import TrainingArguments
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import DataCollatorWithPadding
import evaluate
from transformers import Trainer
import json
from pathlib import Path
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

## Setting Random Seed

In [ ]:
seed = 42

def seed_setup(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_setup(seed)

# Setting device

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Loading data

In [ ]:
ds = load_dataset("google-research-datasets/go_emotions", "simplified")

In [ ]:
label_names = ds['train'].features['labels'].feature.names
label_names_idx = {i:name for i, name in enumerate(ds['train'].features['labels'].feature.names)}
num_labels = len(label_names)

In [ ]:
print(f'Label names: {label_names}, Number of Labels: {num_labels}')

# Loading model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("FacebookAI/xlm-roberta-base")
model = AutoModelForSequenceClassification.from_pretrained("FacebookAI/xlm-roberta-base", num_labels=num_labels, problem_type="multi_label_classification")
model.to(device)

#FacebookAI/roberta-base
#answerdotai/ModernBERT-base
#google-bert/bert-base-uncased
#distilbert/distilbert-base-uncased

# Determining training paramters

In [ ]:
outdir = './finetuned_xml_RoBERTa'

args = TrainingArguments(
    output_dir = outdir,
    num_train_epochs = 10,
    learning_rate = 2e-5,
    logging_steps = 100,
    warmup_steps = 0,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    save_total_limit = 2,
    seed = seed,
    metric_for_best_model="macro_f1",
    load_best_model_at_end=True,
    greater_is_better=True,
    eval_strategy = "epoch",
    save_strategy="epoch",
    push_to_hub = False,
    report_to=[],
)

# Pre-process and tokenize data

In [ ]:
def tokenize(examples):
    tokens = tokenizer(examples["text"], truncation=True, max_length=512)
    return tokens
    
datasets = ds.map(tokenize, batched=True)

def one_hot_labes(batch):
    multi_hot_labels = np.zeros((len(batch['labels']), num_labels), dtype=np.float32)
    
    for i, labels in enumerate(batch['labels']):
        multi_hot_labels[i,labels] = 1
    batch['labels'] = multi_hot_labels.astype(np.float32)
    return batch
        
datasets = datasets.map(one_hot_labes, batched=True)

collator_base = DataCollatorWithPadding(tokenizer=tokenizer)

def collator(features):
    batch = collator_base(features)
    batch['labels'] = batch['labels'].to(torch.float32)
    return batch

# Intializing metrics

In [ ]:
def metrics_calculation(eval_pred):
    logits, labels = eval_pred
    probs = torch.sigmoid(torch.tensor(logits)).numpy()
    preds = (probs >= 0.3).astype(int)
    true_labels = labels.astype(int)

    results = {}
        
    results["accuracy"] = accuracy_score(true_labels, preds)
    results["macro_precision"], results["macro_recall"], results["macro_f1"], _ = precision_recall_fscore_support(true_labels, preds, average="macro", zero_division=0)
    results["micro_precision"], results["micro_recall"], results["micro_f1"], _ = precision_recall_fscore_support(true_labels, preds, average="micro", zero_division=0)
    results["weighted_precision"], results["weighted_recall"], results["weighted_f1"], _ = precision_recall_fscore_support(true_labels, preds, average="weighted", zero_division=0)

    for i in range(num_labels):
        emotion = label_names_idx[i]
        emotion_true = true_labels[:, i]
        emotion_pred = preds[:, i]
        results[emotion + "_accuracy"] = accuracy_score(emotion_true, emotion_pred)
        results[emotion + "_precision"], results[emotion + "_recall"], results[emotion + "_f1"], _ = precision_recall_fscore_support(emotion_true, emotion_pred, average="binary", zero_division=0)
        
    return results

# Intializing Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=datasets["train"],
    eval_dataset=datasets["validation"],
    tokenizer=tokenizer,
    data_collator=collator,
    compute_metrics=metrics_calculation
)

# Train and save model

In [ ]:
trainer.train()
trainer.save_model(outdir)
tokenizer.save_pretrained(outdir)

## Saving traning metrics

In [ ]:
logs = [log for log in trainer.state.log_history if "eval_macro_f1" in log]

with open("training_log__xml_RoBERTa.json", "w") as f:
    json.dump(logs, f, indent=2)

## Testing model

In [ ]:
test_result = trainer.evaluate(eval_dataset=datasets["test"])

In [ ]:
with open("test_log_xml_RoBERTa.json", "w") as f:
    json.dump(test_result, f, indent=2)